In [13]:
!pip install BeautifulSoup4
!pip install requests
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [19]:
# Fijamos url de la web
url = 'https://tarifaluzhora.es/'

# Hacemos la petición a la página
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

,column1
00h - 01h:,0.33468 €/kWh
01h - 02h:,0.31902 €/kWh
02h - 03h:,0.30903 €/kWh
03h - 04h:,0.29593 €/kWh
04h - 05h:,0.29989 €/kWh
05h - 06h:,0.3092 €/kWh
06h - 07h:,0.32472 €/kWh
07h - 08h:,0.32824 €/kWh
08h - 09h:,0.37147 €/kWh
09h - 10h:,0.3673 €/kWh


In [65]:
# Obtenemos las horas
horas = soup.find_all('span', itemprop="description")

# Obtenemos los precios
precios = soup.find_all('span', itemprop="price")

# Obtenemos la fecha
fecha = soup.find('input', {'name': 'date'}).get('value')

# Creamos un array con el contenido de las horas
columnas = []
for h in horas:
    columnas.append(h.text)
    
# Creamos un array con el contenido de los precios
contenido = []
for p in precios:
    contenido.append(p.text)

'30/03/2022'

In [82]:
# Creamos un dataset con los datos del primer día, cuyas columnas son la fecha y las horas
df = pd.DataFrame(data=[np.array(contenido)], columns=columnas)
df['fecha'] = fecha
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df

,fecha,00h - 01h:,01h - 02h:,02h - 03h:,03h - 04h:,04h - 05h:,05h - 06h:,06h - 07h:,07h - 08h:,08h - 09h:,...,14h - 15h:,15h - 16h:,16h - 17h:,17h - 18h:,18h - 19h:,19h - 20h:,20h - 21h:,21h - 22h:,22h - 23h:,23h - 24h:
0,30/03/2022,0.33468 €/kWh,0.31902 €/kWh,0.30903 €/kWh,0.29593 €/kWh,0.29989 €/kWh,0.3092 €/kWh,0.32472 €/kWh,0.32824 €/kWh,0.37147 €/kWh,...,0.33943 €/kWh,0.33612 €/kWh,0.32761 €/kWh,0.31185 €/kWh,0.39006 €/kWh,0.4168 €/kWh,0.43635 €/kWh,0.44541 €/kWh,0.35615 €/kWh,0.35182 €/kWh
